# Kalman Filter - Simple Demo (TensorFlow Version with Standard Notation)

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt

tfd = tfp.distributions

## System Model

### State Space Representation

**State equation:**
$$\mathbf{x}_t = F_t \mathbf{x}_{t-1} + B_t \mathbf{u}_t + \mathbf{w}_t, \quad \mathbf{w}_t \sim \mathcal{N}(\mathbf{0}, Q_t)$$

**Observation equation:**
$$\mathbf{z}_t = H_t \mathbf{x}_t + \mathbf{v}_t, \quad \mathbf{v}_t \sim \mathcal{N}(\mathbf{0}, R_t)$$

Where:
- $\mathbf{x}_t \in \mathbb{R}^{n_x}$ is the latent (hidden) state vector
- $\mathbf{z}_t \in \mathbb{R}^{n_z}$ is the observation vector
- $\mathbf{w}_t, \mathbf{v}_t$ are mutually independent Gaussian noise terms
- $F_t$ is the state transition matrix
- $H_t$ is the observation matrix
- $Q_t$ and $R_t$ are the process and observation covariance matrices

## Initialization

**Initial conditions:**
$$\hat{\mathbf{x}}_{0|0} = \mathbb{E}[\mathbf{x}_0], \qquad \Sigma_{0|0} = \mathrm{Cov}[\mathbf{x}_0]$$

In [ ]:
# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Initial state (真实状态)
x_t = tf.constant([[0.0],
                   [1.0]], dtype=tf.float32)

# Initial posterior estimate (后验估计初始值): x_hat_{0|0}
x_hat_t_t = tf.Variable([[0.0],
                         [1.0]], dtype=tf.float32)

# Process noise covariance (过程噪声协方差): Q_t
Q_t = tf.constant([[0.5, 0.0],
                   [0.0, 0.5]], dtype=tf.float32)

# Measurement noise covariance (测量噪声协方差): R_t
R_t = tf.constant([[0.5, 0.0],
                   [0.0, 0.5]], dtype=tf.float32)

# State transition matrix (系统矩阵): F_t
F_t = tf.constant([[1.0, 1.0],
                   [0.0, 1.0]], dtype=tf.float32)

# Observation matrix (测量矩阵): H_t
H_t = tf.constant([[1.0, 0.0],
                   [0.0, 1.0]], dtype=tf.float32)

# Posterior error covariance (后验误差协方差初始值): Sigma_{0|0}
Sigma_t_t = tf.Variable([[1.0, 0.0],
                         [0.0, 1.0]], dtype=tf.float32)

# Create noise distributions using tfd
# Process noise distribution: w_t ~ N(0, Q_t)
process_noise_dist = tfd.MultivariateNormalFullCovariance(
    loc=tf.zeros([Q_t.shape[0]], dtype=tf.float32),
    covariance_matrix=Q_t
)

# Measurement noise distribution: v_t ~ N(0, R_t)
measurement_noise_dist = tfd.MultivariateNormalFullCovariance(
    loc=tf.zeros([R_t.shape[0]], dtype=tf.float32),
    covariance_matrix=R_t
)

# Storage arrays (用于记录)
x_t_log = tf.Variable(x_t, dtype=tf.float32)
z_t_log = tf.Variable(tf.zeros([2, 0]), dtype=tf.float32)
x_hat_t_t_log = tf.Variable(x_hat_t_t, dtype=tf.float32)
x_hat_t_tm1_log = tf.Variable(tf.zeros([2, 0]), dtype=tf.float32)  # 记录先验估计值

N_step = 30

## Kalman Filter Algorithm

### Prediction Step

**Predicted state estimate:**
$$\hat{\mathbf{x}}_{t|t-1} = F_t \hat{\mathbf{x}}_{t-1|t-1} + B_t \mathbf{u}_t$$

**Predicted error covariance:**
$$\Sigma_{t|t-1} = F_t \Sigma_{t-1|t-1} F_t^\top + Q_t$$

### Update Step

**Innovation (residual):**
$$\mathbf{r}_t = \mathbf{z}_t - H_t \hat{\mathbf{x}}_{t|t-1}$$

**Innovation covariance:**
$$S_t = H_t \Sigma_{t|t-1} H_t^\top + R_t$$

**Kalman gain:**
$$K_t = \Sigma_{t|t-1} H_t^\top S_t^{-1}$$

**Posterior state estimate:**
$$\hat{\mathbf{x}}_{t|t} = \hat{\mathbf{x}}_{t|t-1} + K_t \mathbf{r}_t$$

**Posterior error covariance:**
$$\Sigma_{t|t} = (I - K_t H_t)\Sigma_{t|t-1}$$

In [ ]:
for i in range(N_step):
    # ============================================
    # State Transition: x_t = F_t * x_{t-1} + w_t
    # ============================================
    # Sample process noise from Q_t: w_t ~ N(0, Q_t)
    w_t = tf.reshape(process_noise_dist.sample(), [-1, 1])
    
    x_t = tf.matmul(F_t, x_t) + w_t
    x_t_log = tf.concat([x_t_log, x_t], axis=1)
    
    # ============================================
    # Observation: z_t = H_t * x_t + v_t
    # ============================================
    # Sample measurement noise from R_t: v_t ~ N(0, R_t)
    v_t = tf.reshape(measurement_noise_dist.sample(), [-1, 1])
    
    z_t = tf.matmul(H_t, x_t) + v_t
    z_t_log = tf.concat([z_t_log, z_t], axis=1)
    
    # ============================================
    # PREDICTION STEP
    # ============================================
    # Predicted state estimate: x_hat_{t|t-1} = F_t * x_hat_{t-1|t-1}
    x_hat_t_tm1 = tf.matmul(F_t, x_hat_t_t)
    
    # Predicted error covariance: Sigma_{t|t-1} = F_t * Sigma_{t-1|t-1} * F_t^T + Q_t
    Sigma_t_tm1 = tf.matmul(tf.matmul(F_t, Sigma_t_t), F_t, transpose_b=True) + Q_t
    
    x_hat_t_tm1_log = tf.concat([x_hat_t_tm1_log, x_hat_t_tm1], axis=1)
    
    # ============================================
    # UPDATE STEP
    # ============================================
    # Innovation (residual): r_t = z_t - H_t * x_hat_{t|t-1}
    r_t = z_t - tf.matmul(H_t, x_hat_t_tm1)
    
    # Innovation covariance: S_t = H_t * Sigma_{t|t-1} * H_t^T + R_t
    S_t = tf.matmul(tf.matmul(H_t, Sigma_t_tm1), H_t, transpose_b=True) + R_t
    
    # Kalman gain: K_t = Sigma_{t|t-1} * H_t^T * S_t^{-1}
    K_t = tf.matmul(tf.matmul(Sigma_t_tm1, H_t, transpose_b=True), tf.linalg.inv(S_t))
    
    # Posterior state estimate: x_hat_{t|t} = x_hat_{t|t-1} + K_t * r_t
    x_hat_t_t_new = x_hat_t_tm1 + tf.matmul(K_t, r_t)
    
    # Posterior error covariance: Sigma_{t|t} = (I - K_t * H_t) * Sigma_{t|t-1}
    Sigma_t_t_new = tf.matmul(tf.eye(2, dtype=tf.float32) - tf.matmul(K_t, H_t), Sigma_t_tm1)
    
    x_hat_t_t_log = tf.concat([x_hat_t_t_log, x_hat_t_t_new], axis=1)
    
    # Update variables for next iteration
    x_hat_t_t.assign(x_hat_t_t_new)
    Sigma_t_t.assign(Sigma_t_t_new)

print(f"Kalman filter completed for {N_step} steps")

## Plot Results

In [ ]:
legend_text = ['Real', 'Measure', 'Prior', 'Posterior']
plt.figure(figsize=(8, 6))

# Plot 1: Position
plt.subplot(211)
plt.plot(range(0, N_step+1), x_t_log[0, :].numpy(),
         range(1, N_step+1), z_t_log[0, :].numpy(),
         range(1, N_step+1), x_hat_t_tm1_log[0, :].numpy(),
         range(0, N_step+1), x_hat_t_t_log[0, :].numpy(), marker='.')
plt.legend(legend_text)
plt.xlabel('Time($s$)')
plt.ylabel('Position($m$)')

# Plot 2: Velocity
plt.subplot(212)
plt.plot(range(0, N_step+1), x_t_log[1, :].numpy(),
         range(1, N_step+1), z_t_log[1, :].numpy(),
         range(1, N_step+1), x_hat_t_tm1_log[1, :].numpy(),
         range(0, N_step+1), x_hat_t_t_log[1, :].numpy(), marker='.')
plt.legend(legend_text)
plt.xlabel('Time($s$)')
plt.ylabel('Velocity($m\cdot s^{-1}$)')

plt.tight_layout()
plt.savefig('kalman_filter_third_tensorflow.png', dpi=150, bbox_inches='tight')
plt.show()

print("Plot saved as 'kalman_filter_third_tensorflow.png'")